In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime

In [3]:
data = pd.read_excel('./Attachment 2.xlsx').values
lines = np.unique(np.array([i[0]+i[1]for i in data[:,[1,2]]]))
lines = np.array([[i[0],i[1]] for i in lines])
date = np.unique(data[:,0])

In [5]:
d = np.zeros(shape=(lines.shape[0],date.shape[0]))
d1 = np.zeros(shape=(lines.shape[0],date.shape[0]))
for i in range(d.shape[0]):
    data1 = data[np.logical_and(data[:,1]==lines[i,0],data[:,2]==lines[i,1])]
    for j in range(d.shape[1]):
        data2 = data1[data1[:,0]==date[j]]
        if data2.shape[0]==0:
            d[i,j]=0
            d1[i,j]=0
        else:
            d[i,j]=1
            d1[i,j]=data2[0,-1]


all_pre1 = []

for i in range(lines.shape[0]):
    y = d[i]
# 使用BP神经网络模型来预测每一条线路的货运量
for i in range(lines.shape[0]):
    y = d[i]
    # 将数据划分为训练集和测试集
    train_size = int(len(y) * 0.8)
    train, test = y[:train_size], y[train_size:]

    # 数据归一化
    scaler = MinMaxScaler()
    train = scaler.fit_transform(train.reshape(-1, 1))
    test = scaler.transform(test.reshape(-1, 1))

    # 重塑数据以适应BP神经网络模型的输入格式
    look_back = 7
    X_train, y_train = [], []
    for j in range(look_back, len(train)):
        X_train.append(train[j-look_back:j, 0])
        y_train.append(train[j, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_test, y_test = [], []
    for j in range(look_back, len(test)):
        X_test.append(test[j-look_back:j, 0])
        y_test.append(test[j, 0])
    X_test, y_test = np.array(X_test), np.array(y_test)

    # 创建BP神经网络模型
    model = Sequential()
    model.add(Dense(64, input_dim=look_back, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # 训练模型
    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

    # 预测
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)

    # 将预测结果转换回原始尺度
    train_predict = scaler.inverse_transform(train_predict)
    y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
    test_predict = scaler.inverse_transform(test_predict)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # 计算预测准确性，例如使用均方误差（MSE）作为评估指标
    train_mse = np.mean((train_predict - y_train) ** 2)
    test_mse = np.mean((test_predict - y_test) ** 2)

    # 计算预测日期与最后一个训练日期之间的天数
    last_train_date = datetime.date(2023, 4, 27)
    start_pred_date = datetime.date(2023, 4, 28)
    end_pred_date = datetime.date(2023, 4, 30)
    days_to_predict = (end_pred_date-start_pred_date).days

    # 使用训练数据的最后一部分来开始预测
    input_data = train[-look_back:]

    predictions = []

    # 预测每一天的货量
    for i in range(days_to_predict):
        input_data_reshaped = input_data.reshape(1, look_back)
        pred = model.predict(input_data_reshaped)
        predictions.append(pred[0,0])

        # 更新输入数据，用预测值替换最早的值
        input_data = np.roll(input_data, -1)
        input_data[-1] = pred

    # 将预测值转换回原始尺度
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    tdays = []
    # 打印预测结果
    for i, pred in enumerate(predictions, start=1):
        pred_date = start_pred_date + datetime.timedelta(days=i - 1)
        tdays += [pred[0]]
    all_pre1+=[tdays]
all_pre1 = np.array(all_pre1)

Epoch 1/50
41/41 [==============================] - 0s 818us/step - loss: 0.2006
Epoch 2/50
41/41 [==============================] - 0s 766us/step - loss: 0.0487
Epoch 3/50
41/41 [==============================] - 0s 947us/step - loss: 0.0455
Epoch 4/50
41/41 [==============================] - 0s 773us/step - loss: 0.0444
Epoch 5/50
41/41 [==============================] - 0s 947us/step - loss: 0.0428
Epoch 6/50
41/41 [==============================] - 0s 923us/step - loss: 0.0420
Epoch 7/50
41/41 [==============================] - 0s 848us/step - loss: 0.0433
Epoch 8/50
41/41 [==============================] - 0s 773us/step - loss: 0.0412
Epoch 9/50
41/41 [==============================] - 0s 1ms/step - loss: 0.0408
Epoch 10/50
41/41 [==============================] - 0s 898us/step - loss: 0.0412
Epoch 11/50
41/41 [==============================] - 0s 723us/step - loss: 0.0419
Epoch 12/50
41/41 [==============================] - 0s 898us/step - loss: 0.0410
Epoch 13/50
41/41 [========

In [6]:
all_pre1

array([[ 0.0203664 ,  0.03223394],
       [ 0.9672402 ,  0.95368874],
       [ 0.60197437,  0.7448534 ],
       [ 0.6559913 ,  0.6281353 ],
       [ 0.01429183,  0.02836278],
       [ 0.93220544,  0.9216571 ],
       [ 0.04286176,  0.0504597 ],
       [ 0.01490689,  0.0291424 ],
       [ 0.8700354 ,  0.86126393],
       [ 0.04702459,  0.08460042],
       [ 0.01612487,  0.02138446],
       [ 0.01655643,  0.02526362],
       [ 0.97857076,  0.9719043 ],
       [ 0.673841  ,  0.24057253],
       [ 0.01302513,  0.02002681],
       [ 0.94409347,  0.929822  ],
       [ 0.9483658 ,  0.92646366],
       [ 1.0132676 ,  1.0253035 ],
       [ 0.9745445 ,  0.97047037],
       [ 0.96588224,  0.9636049 ],
       [ 0.9307744 ,  0.9045832 ],
       [ 0.98772573,  0.98192155],
       [ 0.01886857,  0.04074421],
       [ 0.01045539,  0.01519404],
       [ 0.02768662,  0.06038927],
       [ 0.01539247,  0.02833703],
       [ 0.9943963 ,  0.9934871 ],
       [ 0.02159583,  0.0397015 ],
       [ 0.88226867,

In [8]:
all_pre = []
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), 0])
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)

for i in range(lines.shape[0]):
    y = d1[i]
    y[y==0]=y.mean()
    # 将数据划分为训练集和测试集
    train_size = int(len(y) * 0.8)
    train, test = y[:train_size], y[train_size:]

    # 数据归一化
    scaler = MinMaxScaler()
    train = scaler.fit_transform(train.reshape(-1, 1))
    test = scaler.transform(test.reshape(-1, 1))

    # 创建数据生成器
    look_back = 7
    X_train, y_train = create_dataset(train, look_back)
    X_test, y_test = create_dataset(test, look_back)

    # 创建BP神经网络模型
    model = Sequential()
    model.add(Dense(64, input_dim=look_back, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # 训练模型
    model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=1)

    # 预测
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)

    # 将预测结果转换回原始尺度
    train_predict = scaler.inverse_transform(train_predict)
    y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
    test_predict = scaler.inverse_transform(test_predict)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # 计算预测准确性，例如使用均方误差（MSE）作为评估指标
    train_mse = np.mean((train_predict - y_train) ** 2)
    test_mse = np.mean((test_predict - y_test) ** 2)

    # 计算预测日期与最后一个训练日期之间的天数
    last_train_date = datetime.date(2023, 4, 27)
    start_pred_date = datetime.date(2023, 4, 28)
    end_pred_date = datetime.date(2023, 4, 29)
    days_to_predict = (end_pred_date-start_pred_date).days

    # 使用训练数据的最后一部分来开始预测
    input_data = train[-look_back:]

    predictions = []

    # 预测每一天的货量
    for i in range(days_to_predict):
        input_data_reshaped = input_data.reshape(1, look_back)
        pred = model.predict(input_data_reshaped)
        predictions.append(pred[0, 0])

        # 更新输入数据，用预测值替换最早的值
        input_data = np.roll(input_data, -1)
        input_data[-1] = pred

    # 将预测值转换回原始尺度
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    tdays = []
    # 打印
    for i, pred in enumerate(predictions, start=1):
        pred_date = start_pred_date + datetime.timedelta(days=i - 1)
        tdays += [pred[0]]
    all_pre+=[tdays]
all_pre = np.array(all_pre)

Epoch 1/20
650/650 [==============================] - 1s 751us/step - loss: 0.0102
Epoch 2/20
650/650 [==============================] - 0s 715us/step - loss: 0.0082
Epoch 3/20
650/650 [==============================] - 0s 700us/step - loss: 0.0080
Epoch 4/20
650/650 [==============================] - 0s 696us/step - loss: 0.0075
Epoch 5/20
650/650 [==============================] - 0s 647us/step - loss: 0.0074
Epoch 6/20
650/650 [==============================] - 0s 650us/step - loss: 0.0071
Epoch 7/20
650/650 [==============================] - 0s 654us/step - loss: 0.0070
Epoch 8/20
650/650 [==============================] - 0s 635us/step - loss: 0.0069
Epoch 9/20
650/650 [==============================] - 0s 660us/step - loss: 0.0068
Epoch 10/20
650/650 [==============================] - 0s 653us/step - loss: 0.0068
Epoch 11/20
650/650 [==============================] - 0s 660us/step - loss: 0.0068
Epoch 12/20
650/650 [==============================] - 0s 668us/step - loss: 0.0067
E

In [9]:
all_pre1[all_pre1>0.5]=1
all_pre1[all_pre1<=0.5]=0

In [10]:
p2 = all_pre*all_pre1

In [11]:
p2

array([[  0.      ,   0.      ],
       [ 69.32092 ,  69.32092 ],
       [ 54.88627 ,  54.88627 ],
       [ 42.259777,  42.259777],
       [  0.      ,   0.      ],
       [ 52.39557 ,  52.39557 ],
       [  0.      ,   0.      ],
       [  0.      ,   0.      ],
       [ 39.567245,  39.567245],
       [  0.      ,   0.      ],
       [  0.      ,   0.      ],
       [  0.      ,   0.      ],
       [141.22119 , 141.22119 ],
       [ 41.367012,   0.      ],
       [  0.      ,   0.      ],
       [ 59.490124,  59.490124],
       [ 65.896774,  65.896774],
       [637.42194 , 637.42194 ],
       [ 50.375946,  50.375946],
       [ 71.20612 ,  71.20612 ],
       [ 61.618633,  61.618633],
       [ 49.661015,  49.661015],
       [  0.      ,   0.      ],
       [  0.      ,   0.      ],
       [  0.      ,   0.      ],
       [  0.      ,   0.      ],
       [ 46.301655,  46.301655],
       [  0.      ,   0.      ],
       [217.27121 , 217.27121 ],
       [100.20468 , 100.20468 ],
       [12

In [12]:
all_pre1

array([[0., 0.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [1., 0.],
       [0., 0.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.

In [13]:
pd.DataFrame(np.c_["1",lines,all_pre1,p2],columns=["起点","终点","28号是否开通","29号是否开通","28号预测值","29号预测值"]).to_excel("q3result.xlsx")

In [16]:
df = pd.read_excel('./q3result.xlsx')

I-S
M-G
S-Q
V-A
Y-L
D-R
J-K
Q-O
U-O
Y-W


In [17]:
# 定义路线列表
routes = [['I', 'S'], ['M', 'G'], ['S', 'Q'], ['V', 'A'], ['Y', 'L'], ['D', 'R'], ['J', 'K'], ['Q', 'O'], ['U', 'O'], ['Y', 'W']]

# 创建新的 DataFrame
new_df = pd.DataFrame(columns=df.columns)

# 遍历原始 DataFrame 中的每一行数据
for i, row in df.iterrows():
    # 获取起点和终点城市的名称
    start, end = row["起点"], row["终点"]

    # 判断起点和终点城市是否在路线列表中
    if [start, end] in routes:
        # 将该行数据添加到新的 DataFrame 中
        new_df = new_df.append(row)

E:\Users\panyue\AppData\Local\Temp\ipykernel_16316\2280098619.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
E:\Users\panyue\AppData\Local\Temp\ipykernel_16316\2280098619.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
E:\Users\panyue\AppData\Local\Temp\ipykernel_16316\2280098619.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
E:\Users\panyue\AppData\Local\Temp\ipykernel_16316\2280098619.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
E:\Users\panyue\AppData\Local\Temp\ipykernel_16316\2280098619.py:15: Fut

In [19]:
new_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [20]:
new_df

,起点,终点,28号是否开通,29号是否开通,28号预测值,29号预测值
9,D,R,0.0,0.0,0.000000,0.000000
25,I,S,0.0,0.0,0.000000,0.000000
28,J,K,1.0,1.0,217.271210,217.271210
44,M,G,1.0,1.0,43.165120,43.165120
58,Q,O,0.0,0.0,0.000000,0.000000
68,S,Q,1.0,1.0,85.354370,85.354370
72,U,O,0.0,0.0,0.000000,0.000000
74,V,A,1.0,1.0,113.272160,113.272160
87,Y,L,1.0,1.0,38.363564,38.363564
88,Y,W,0.0,0.0,0.000000,0.000000
